# New York taxis trips

This homework is about New York taxi trips. Here is something from [Todd Schneider](https://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/):

> The New York City Taxi & Limousine Commission has released a  detailed historical dataset covering over 1 billion individual taxi trips in the city from January 2009 through December 2019. 
Taken as a whole, the detailed trip-level data is more than just a vast list of taxi pickup and drop off coordinates: it's a story of a City. 
How bad is the rush hour traffic from Midtown to JFK? 
Where does the Bridge and Tunnel crowd hang out on Saturday nights?
What time do investment bankers get to work? How has Uber changed the landscape for taxis?
The dataset addresses all of these questions and many more.

The NY taxi trips dataset has been plowed by series of distinguished data scientists.
The dataset is available from on Amazon S3 (Amazon's cloud storage service).
The link for each file has the following form:

    https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_{year}-{month}.csv

There is one CSV file for each NY taxi service (`yellow`, `green`, `fhv`) and each calendar month (replacing `{year}` and `{month}` by the desired ones).
Each file is moderately large, a few gigabytes. 
The full dataset is relatively large if it has to be handled on a laptop (several hundred gigabytes).

You will focus on the `yellow` taxi service and a pair of months, from year 2015 and from year 2018. 
Between those two years, for hire vehicles services have taken off and carved a huge marketshare.

Whatever the framework you use, `CSV` files prove hard to handle. 
After downloading the appropriate files (this takes time, but this is routine), a first step will consist in converting the csv files into a more Spark friendly format such as `parquet`.

Saving into one of those formats require decisions about bucketing, partitioning and so on. Such decisions influence performance. It is your call.
Many people have been working on this dataset, to cite but a few:


- [1 billion trips with a vengeance](https://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/)
- [1 billion trips with R and SQL ](http://freerangestats.info/blog/2019/12/22/nyc-taxis-sql)
- [1 billion trips with redshift](https://tech.marksblogg.com/billion-nyc-taxi-rides-redshift.html)
- [nyc-taxi](https://github.com/fmaletski/nyc-taxi-map)

Depending on your internet connection, **download the files** corresponding to **"yellow" taxis** for the years 2015 and 2018. Download **at least one month** (the same) for 2015 and 2018, if you can download all of them.

**Hint.** The 12 csv for 2015 are about 23GB in total, but the corresponding parquet file, if you can create it for all 12 months, is only about 3GB.

You **might** need the following stuff in order to work with GPS coordinates and to plot things easily.

In [1]:
!pip install geojson geopandas plotly geopy

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install ipyleaflet

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


For this homework **we will let you decide on the tools to use** (expected for Spark) and to **find out information all by yourself** (but don't hesitate to ask questions on the `slack` channel).

# Loading data as parquet files

We want to organize the data on a per year and per service basis. 
We want to end up with one `parquet` file for each year and each taxi service, since parquet is much better than CSV files.

**Hint.** Depending on your internet connection and your laptop, you can use only the "yellow" service and use one month of 2015 and 2018

CSV files can contain corrupted lines. You may have to work in order to perform ETL (Extract-Transform-Load) in order obtain a properly typed data frame.

You are invited to proceed as follows:

1. Try to read the CSV file without imposing a schema. 
1. Inspect the inferred schema. Do you agree with Spark's typing decision?
1. Eventually correct the schema and read again the data
1. Save the data into parquet files
1. In the rest of your work, **you will only use the parquet files you created**, not the csv files (don't forget to choose a partitioning column and a number of partitions when creating the parquet files).

**Hint.** Don't forget to ask `Spark` to use all the memory and ressources from your computer.

**Hint.** Don't foreget that you should specify a partitioning column and a number of partitions when creating the parquet files.

**Hint.** Note that the schemas of the 2015 and 2018 data are different...

**Hint.** When working on this, ask you and answer to the following questions:

1. What is the `StorageLevel` of the dataframe after reading the csv files?
1. What is the number of partitions of the dataframe? 
1. Is it possible to tune this number at loading time? 
1. Why would we want to modify the number of partitions when creating the parquet files?

In [3]:
# import the usual suspects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import sys
import timeit
import plotly.graph_objects as go

%matplotlib inline
import seaborn as sns

sns.set_context("notebook", font_scale=1.2)

In [4]:
# spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
from pyspark.sql.catalog import Catalog
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType

In [6]:
# Start the SparkSession
conf = SparkConf().setAppName("Spark SQL Illustrations")
sc = SparkContext(conf=conf)

spark = (SparkSession
    .builder
    .appName("New York taxis trips")
    .getOrCreate()
)

1.Try to read the CSV file without imposing a schema.

In [7]:
import requests
from pathlib import Path

# download the data of 2015
path2015 = Path('yellow_tripdata_2015-07.csv')
if not path2015.exists():
    url = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2015-07.csv"
    r = requests.get(url)
    with open(os.path.join('./', 'yellow_tripdata_2015-07.csv'), 'wb') as f:
        f.write(r.content)

In [8]:
# download the data of 2018
path2018 = Path('yellow_tripdata_2018-07.csv')
if not path2018.exists():
    url = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-07.csv"
    r = requests.get(url)
    with open(os.path.join('./', 'yellow_tripdata_2018-07.csv'), 'wb') as f:
        f.write(r.content)

In [9]:
# Load data from a csv file
df_sp2015 = spark.read\
             .format('csv')\
             .option("header", "true")\
             .option("mode", "FAILFAST")\
             .option("inferSchema", "true")\
             .option("sep", ",")\
             .load("yellow_tripdata_2015-07.csv")

df_sp2015.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [10]:
df_sp2018 = spark.read\
             .format('csv')\
             .option("header", "true")\
             .option("mode", "FAILFAST")\
             .option("inferSchema", "true")\
             .option("sep", ",")\
             .load("yellow_tripdata_2018-07.csv")

df_sp2018.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



2.Inspect the inferred schema. Do you agree with Spark's typing decision?

Non, les colonnes "tpep_pickup_datetime" et "tpep_dropoff_datetime" doivent être de type date et pas de type string.

3.Eventually correct the schema and read again the data

In [11]:
df_sp2015 = df_sp2015\
            .withColumn("tpep_pickup_datetime",(fn.to_timestamp(col("tpep_pickup_datetime"))))\
            .withColumn("tpep_dropoff_datetime",(fn.to_timestamp(col("tpep_dropoff_datetime"))))

df_sp2015.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [12]:
df_sp2018 = df_sp2018\
            .withColumn("tpep_pickup_datetime",(fn.to_timestamp(col("tpep_pickup_datetime"))))\
            .withColumn("tpep_dropoff_datetime",(fn.to_timestamp(col("tpep_dropoff_datetime"))))

df_sp2018.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



4.Save the data into parquet files

In [14]:
# clean data df_sp2015
!pip install shapely

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
import geopandas
import shapely.geometry
shp_path = os.path.join('./', 'nyc-taxi-zone.shp')
nyc_taxi_zones = geopandas.GeoDataFrame.from_file(shp_path)
print(nyc_taxi_zones.head(20))

          borough  location_i  objectid  shape_area  shape_leng  \
0             EWR         1.0       1.0    0.000782    0.116357   
1          Queens         2.0       2.0    0.004866    0.433470   
2           Bronx         3.0       3.0    0.000314    0.084341   
3       Manhattan         4.0       4.0    0.000112    0.043567   
4   Staten Island         5.0       5.0    0.000498    0.092146   
5   Staten Island         6.0       6.0    0.000606    0.150491   
6          Queens         7.0       7.0    0.000390    0.107417   
7          Queens         8.0       8.0    0.000027    0.027591   
8          Queens         9.0       9.0    0.000338    0.099784   
9       Manhattan        24.0      24.0    0.000061    0.047000   
10         Queens        10.0      10.0    0.000436    0.099839   
11       Brooklyn        11.0      11.0    0.000265    0.079211   
12      Manhattan        12.0      12.0    0.000042    0.036661   
13      Manhattan        13.0      13.0    0.000149    0.05028

In [ ]:
def in_area(longitude, latitude):
    point = shapely.geometry.Point(longitude, latitude)
    for i in range(len(nyc_taxi_zones)):
        if(point.within(nyc_taxi_zones.iloc[i]['geometry'])):
            return int(nyc_taxi_zones.iloc[i]['location_i'])    
    return 0

udf_in_area = fn.udf(lambda x,y : in_area(x,y), IntegerType())

# ajouter la colonne de domaine
def add_area(df):
    df = df\
        .withColumn('PULocationID', udf_in_area(col('pickup_longitude'), col('pickup_latitude')))\
        .withColumn('DOLocationID', udf_in_area(col('dropoff_longitude'), col('dropoff_latitude')))
    return df

#df = df2015_f.limit(1)
#print(in_area(-74.00090026855469,40.73073196411133))
df_sp2015 = add_area(df_sp2015).toPandas()
df_sp2015.head()
#df_sp2015 = add_area(df_sp2015)
#df_sp2015.show()

In [ ]:
nyc_taxi_zones = nyc_taxi_zones[['LocationID','zone','borough','lon','lat']]

In [ ]:
df201507_pickup = pd.merge(left=df_sp2015,right=nyc_taxi_zones,how='inner',left_on='PULocationID',right_on='LocationID')
df201507_pickup['pickup_longitude'] = df201507_pickup['lon']
df201507_pickup['pickup_latitude'] = df201507_pickup['lat']
df201507_pickup['pickup_zone'] = df201507_pickup['zone']
df201507_pickup['pickup_borough'] = df201507_pickup['borough']
df201507_pickup.head()

In [ ]:
pick_up = list(df201507_pickup.columns)
pick_up.remove('LocationID')
pick_up.remove('zone')
pick_up.remove('borough')
pick_up.remove('lon')
pick_up.remove('lat')
print(pick_up)

In [ ]:
df_sp2015 = df201507_pickup[pick_up]

In [ ]:
df201507_dropoff = pd.merge(left=df_sp2015,right=nyc_taxi_zones,how='inner',left_on='DOLocationID',right_on='LocationID')
df201507_dropoff['dropoff_longitude'] = df201507_dropoff['lon']
df201507_dropoff['dropoff_latitude'] = df201507_dropoff['lat']
df201507_dropoff['dropoff_zone'] = df201507_dropoff['zone']
df201507_dropoff['dropoff_borough'] = df201507_dropoff['borough']
df201507_dropoff.head()

In [ ]:
dropoff = list(df201507_dropoff.columns)
dropoff.remove('LocationID')
dropoff.remove('zone')
dropoff.remove('borough')
dropoff.remove('lon')
dropoff.remove('lat')
print(dropoff)

In [ ]:
df_sp2015 = df201507_dropoff[dropoff]

In [ ]:
df_sp2018 = df_sp2018.toPandas()

In [ ]:
df201807_pickup = pd.merge(left=df_sp2018,right=nyc_taxi_zones,how='inner',left_on='PULocationID',right_on='LocationID')
df201807_pickup['pickup_longitude'] = df201807_pickup['lon']
df201807_pickup['pickup_latitude'] = df201807_pickup['lat']
df201807_pickup['pickup_zone'] = df201807_pickup['zone']
df201807_pickup['pickup_borough'] = df201807_pickup['borough']
df201807_pickup.head()

In [ ]:
pick_up = list(df201807_pickup.columns)
pick_up.remove('LocationID')
pick_up.remove('zone')
pick_up.remove('borough')
pick_up.remove('lon')
pick_up.remove('lat')
print(pick_up)

In [ ]:
df_sp2018 = df201807_pickup[pick_up]

In [ ]:
df201807_dropoff = pd.merge(left=df_sp2018,right=nyc_taxi_zones,how='inner',left_on='DOLocationID',right_on='LocationID')
df201807_dropoff['dropoff_longitude'] = df201807_dropoff['lon']
df201807_dropoff['dropoff_latitude'] = df201807_dropoff['lat']
df201807_dropoff['dropoff_zone'] = df201807_dropoff['zone']
df201807_dropoff['dropoff_borough'] = df201807_dropoff['borough']
df201807_dropoff.head()

In [ ]:
df_sp2018 = df201807_dropoff[dropoff]
df_sp2018.head()

In [ ]:
df_sp2015 = spark.createDataFrame(df_sp2015)
df_sp2018 = spark.createDataFrame(df_sp2018)

In [ ]:
# set the number of partitions
spark.conf.set("spark.sql.shuffle.partitions", 40)

In [ ]:
df_sp2015.write.mode('overwrite').partitionBy('VendorID').parquet("yellow_tripdata_2015-07.parquet")
df_sp2015.rdd.getNumPartitions()

In [ ]:
df_sp2018.write.mode('overwrite').partitionBy('VendorID').parquet("yellow_tripdata_2018-07.parquet")
df_sp2018.rdd.getNumPartitions()

In [ ]:
df2015 = spark.read.parquet("yellow_tripdata_2015-07.parquet")

df2015.show(1)

In [ ]:
df2018 = spark.read.parquet("yellow_tripdata_2018-07.parquet")

df2018.show(1)

# Investigate (at least) one month of data in 2015

From now on, you will be using **the parquet files you created for 2015**.

We shall visualize several features of taxi traffic during one calendar month
in 2015 and the same calendar month in 2018.

**Hint.** In order to build appealing graphics, you may stick to `matplotlib + seaborn`, you can use also
`plotly`, which is used a lot to build interactive graphics, but you can use whatever you want.

The following longitudes and lattitudes encompass Newark and JFK airports, Northern Manhattan and Verazzano bridge.

In [5]:
long_min = -74.10
long_max = -73.70
lat_min = 40.58
lat_max = 40.90

1. Using these boundaries, **filter the 2015 data** (using pickup and dropoff longitude and latitude) and count the number of trips for each value of `passenger_count` and make a plot of that.

In [ ]:
df2015_f = df2015.filter((col("pickup_longitude").between(long_min, long_max)) & (col("pickup_latitude").between(lat_min, lat_max)) & (col("dropoff_longitude").between(long_min, long_max)) & (col("dropoff_latitude").between(lat_min, lat_max)))
df2018_f = df2018.filter((col("pickup_longitude").between(long_min, long_max)) & (col("pickup_latitude").between(lat_min, lat_max)) & (col("dropoff_longitude").between(long_min, long_max)) & (col("dropoff_latitude").between(lat_min, lat_max)))

num_trips = df2015_f\
    .groupBy('passenger_count')\
    .count()\
    .orderBy("passenger_count")
    
num_trips.show()
num_trips = num_trips.toPandas()

In [ ]:
import plotly.express as px

fig = px.bar(num_trips, 
             x="passenger_count", 
             y="count", 
             title="number of trips for each value of passenger_count for July 2015")
fig.show()

Trips with $0$ or larger than $7$ passengers are pretty rare.
We suspect these to be outliers. 
We need to explore these trips further in order order to understand what might be wrong
with them

1. What's special with trips with zero passengers?
1. What's special with trips with more than $6$ passengers?
1. What is the largest distance travelled during this month? Is it the first taxi on the moon?
1. Plot the distribution of the `trip_distance` (using an histogram for instance) during year 2105. Focus on trips with non-zero trip distance and trip distance less than 30 miles.

In [ ]:
from pyspark.sql.functions import dayofweek

df2015_f = df2015_f.withColumn('dayofweek',dayofweek(df2015_f['tpep_pickup_datetime']))
df2015_f.show()

In [ ]:
#What's special with trips with zero passengers?
zero_passengers = df2015_f.filter(col("passenger_count") == 0 ).select('*').toPandas()
zero_passengers.describe()

In [ ]:
zero_passengers.head()

In [ ]:
#What's special with trips with more than  6  passengers?
sixplus_passengers = df2015_f.filter(col("passenger_count") > 6 ).select('*').toPandas()
sixplus_passengers.describe()

In [ ]:
sixplus_passengers.head()

In [ ]:
#What is the largest distance travelled during this month? Is it the first taxi on the moon?
#La distance maximale est de 1.0083357E7, ce qui s'est produit le 2015-07-31 10:35:58 et n'est clairement pas le premier TAXI du mois.

from pyspark.sql.functions import max

df_distance = df2015_f.select('trip_distance','tpep_pickup_datetime').orderBy(df2015_f['trip_distance'].desc())
df_distance.show()

In [ ]:
trip_distance = df2015_f.where('trip_distance > 0 and trip_distance < 30').select('trip_distance','tpep_pickup_datetime','pickup_longitude','pickup_latitude','dayofweek','tip_amount','total_amount')

In [ ]:
trip_distance.count()

In [ ]:
trip_distance.show()

In [ ]:
from pyspark.sql.functions import ceil

trip_distance_f = trip_distance.withColumn('trip_distance_int',ceil(trip_distance['trip_distance']))
trip_distance_f.show()

In [ ]:
distance_histogram = trip_distance_f.groupBy('trip_distance_int').count()
distance_histogram.show()

In [ ]:
distance_histogram = distance_histogram.orderBy(distance_histogram['trip_distance_int']).select('*').toPandas()

fig = px.bar(distance_histogram, 
             x="trip_distance_int", 
             y="count", 
             title="the distribution of the trip_distance")
fig.show()

Let's look at what Spark does for these computations

1. Use the `explain` method or have a look at the [Spark UI](http://localhost:4040/SQL/) to analyze the job. You should be able to assess 
    - Parsed Logical Plan
    - Analyzed Logical Plan
    - Optimized Logical Plan
    - Physical Plan
1. Do the Analyzed Logical Plan and Optimized Logical Plan differ? Spot the differences if any. How would a RDBMS proceed with such a query?

The Analyzed Logical Plan and Optimized Logical Plan are two different phases of the Spark execution plan.The Optimised Logical Plan follows the Analyzed Logical Plan stage and further optimises the output plan from the Analyzed Logical Plan stage.
The Analyzed Logical Plan phase loads the external RDD, then serialises the fields, lists the names of the mapped tables, confirms that the tables exist, and then filters by condition to get the result set.
The Optimized Logical Plan stage optimises the filtering criteria output from the Analyzed Logical Plan stage to determine if it is null (neither hive nor relational databases have this), the difference between this and RDD is that rdd loads all the data in, whereas sparksql stops directly if it encounters a null value.


1. How does the physical plan differ from the Optimized Logical Plan? What are the keywords you would not expects in a RDBMS? What is their meaning? 

the physical plan belongs to the physical execution plan phase, which converts the upstream logical execution plan into a physical execution plan.


1. Inspect the stages on [Spark UI](http://localhost:4040/stages/stage). How many *stages* are necessary to complete the Spark job? What are the roles of `HashAggregate` and `Exchange hashpartitioning`?

With Hash aggregation, the database calculates the hash value based on the value following the group by field and maintains the corresponding list in memory based on the aggregation function used earlier. If there are two aggregate functions after select, then two corresponding data are maintained in memory. Similarly, having n aggregation functions will maintain n of the same arrays.

Exchange hashpartitioning


1. Does the physical plan perform `shuffle` operations? If yes how many?

Any operator with reduce requires a shuffle operation.

1. What are tasks with respect to stages (in Spark language)? How many tasks are your stages made of?

Now, compute the following and produce relevant plots:

1. Break down the trip distance distribution for each day of week
1. Count the number of distinct pickup location
1. Compute and display tips and profits as a function of the pickup location

In [ ]:
#1. Break down the trip distance distribution for each day of week
trip_distance_weekofday = trip_distance_f.groupby('dayofweek','trip_distance_int').count()
trip_distance_weekofday.show()

In [ ]:
trip_distance = trip_distance_weekofday.orderBy(trip_distance_weekofday['trip_distance_int'],trip_distance_weekofday['dayofweek']).select('*').toPandas()

In [ ]:
fig = px.histogram(trip_distance, x="trip_distance_int", y="count", color="dayofweek",nbins=30,title="the trip distance distribution of day of week")
fig.show()

In [ ]:
#2.Count the number of distinct pickup location
trip_distance_f.select('pickup_longitude','pickup_latitude').distinct().count()

In [ ]:
#3.Compute and display tips and profits as a function of the pickup location
pickup_df = trip_distance_f.sample(False,0.0001).toPandas()
pickup_df.head()

In [ ]:
fig = px.scatter(
    pickup_df,
    x = "pickup_longitude", y="pickup_latitude", opacity=0.5,size='tip_amount',title = "The function of tips and pickup location")
fig.show()

In [ ]:
fig = px.scatter(
    pickup_df,
    x = "pickup_longitude", y="pickup_latitude", opacity=0.5,size='total_amount',title = "The profit of tips and pickup location")
fig.show()

# Investigate one month of trips data in 2015 and 2018

 Consider one month of trips data from `yellow` taxis for each year

1. Filter and cache/persist the result

## Assessing seasonalities and looking at time series

Compute and plot the following time series indexed by day of the week and hour of day:

1. The number of pickups
1. The average fare
1. The average trip duration
1. Plot the average of ongoing trips

## Rides to the airports

In order to find the longitude and lattitude of JFK and Newark airport as well as the longitude and magnitudes 
of Manhattan, you can use a service like [geojson.io](http://geojson.io/).
Plot the following time series, indexed the day of the week and hour of the day

1. Median duration of taxi trip leaving Midtown (Southern Manhattan) headed for JFK Airport
1. Median taxi duration of trip leaving from JFK Airport to Midtown (Southern Manhattan)

## Geographic information

For this, you will need to find tools to display maps and to build choropeth maps.
We let you look and find relevant tools to do this.

1. Build a heatmap where color is a function of
    1. number of `pickups`
    2. number of `dropoffs`
    3. number of `pickups` with dropoff at some airport (JFK, LaGuardia, Newark)
2. Build a choropeth map where color is a function of
    1. number of pickups in the area
    1. ratio of number of payments by card/number of cash payments for pickups in the area
    2. ratio of total fare/trip duration for dropoff in the area
3. Build an interactive chorophet with a slider allowing the user to select an `hour of day` and where the color is a function of
    1. average number of dropoffs in the area during that hour the day
    2. average ratio of tip over total fare amount for pickups in the area at given hour of the day

In [ ]:
#4.3.2
import json

with open('NYC Taxi Zones.geojson') as f:
    nyc_geo = json.load(f)

In [ ]:
nyc_geo

In [ ]:
#number of pickups in the area 2015
pickup_zone = df2015.groupby('pickup_zone').count()
pickup_zone.show()

In [ ]:
pickup_zone = pickup_zone.toPandas()
pickup_zone.head()

In [ ]:
chor_pickup = go.Figure(
    go.Choroplethmapbox(
        geojson = nyc_geo,
        featureidkey="properties.zone",
        locations = pickup_zone['pickup_zone'],
        z = pickup_zone['count'],
        zauto=True,
        colorscale='ylorrd',
        marker={'opacity':0.8,'line_width':0.5},
        hovertext = pickup_zone['pickup_zone'],
        hoverinfo='text + z',
        showscale=True,
    )
)

chor_pickup.update_layout( mapbox = {'accesstoken':token,'center':{'lon':-73.965691,'lat':40.97},'zoom':5},margin = {'l':1,'r':1,'t':1,'b':1})
chor_pickup.show()

In [ ]:
#ratio of number of payments by card/number of cash payments for pickups in the area
ratiopay_zone = df2015.groupby('pickup_zone','payment_type').count().toPandas()
ratiopay_zone.head()

In [ ]:
#payment_type = 1 card payment, payment_type = 2 cash payment.
card_payment = ratiopay_zone[ratiopay_zone['payment_type'] == 1]

cash_payment = ratiopay_zone[ratiopay_zone['payment_type'] == 2]

payment_df = pd.merge(card_payment,cash_payment,how='inner',on='pickup_zone')
payment_df.head()

In [ ]:
payment_df['ratio'] = payment_df['count_x']/payment_df['count_y']
payment_df.head()

In [ ]:
payment_pickup = go.Figure(
    go.Choroplethmapbox(
        geojson = nyc_geo,
        featureidkey="properties.zone",
        locations = payment_df['pickup_zone'],
        z = payment_df['ratio'],
        zauto=True,
        colorscale='ylorrd',
        marker={'opacity':0.8,'line_width':0.5},
        hovertext = payment_df['pickup_zone'],
        hoverinfo='text + z',
        showscale=True,
    )
)



payment_pickup.update_layout( mapbox = {'accesstoken':token,'center':{'lon':-73.965691,'lat':40.97},'zoom':5},margin = {'l':1,'r':1,'t':1,'b':1})
payment_pickup.show()

In [ ]:
#ratio of total fare/trip duration for dropoff in the area
from pyspark.sql.functions import dayofweek,hour,date_format,unix_timestamp,sum


ratiofd_zone = df2015.withColumn('trip_duration',(unix_timestamp(df2015['tpep_dropoff_datetime']) - unix_timestamp(df2015['tpep_pickup_datetime'])))




ratiofd_zone = ratiofd_zone.groupby('dropoff_zone').agg(sum(ratiofd_zone['trip_duration']),sum(ratiofd_zone['fare_amount'])).toPandas()
ratiofd_zone.head()

In [ ]:
ratiofd_zone['ratio'] = ratiofd_zone['sum(fare_amount)']/ratiofd_zone['sum(trip_duration)']
ratiofd_zone.head()

In [ ]:
ratiofd_dropoff = go.Figure(
    go.Choroplethmapbox(
        geojson = nyc_geo,
        featureidkey="properties.zone",
        locations = ratiofd_zone['dropoff_zone'],
        z = ratiofd_zone['ratio'],
        zauto=True,
        colorscale='ylorrd',
        marker={'opacity':0.8,'line_width':0.5},
        hovertext = ratiofd_zone['dropoff_zone'],
        hoverinfo='text + z',
        showscale=True,
    )
)



ratiofd_dropoff.update_layout( mapbox = {'accesstoken':token,'center':{'lon':-73.965691,'lat':40.97},'zoom':5},margin = {'l':1,'r':1,'t':1,'b':1})
ratiofd_dropoff.show()

In [ ]:
#4.3.3
from pyspark.sql.functions import hour,date_format


df2015 = df2015.withColumn('hour_pickup',hour(df2015['tpep_pickup_datetime']))
df2015 = df2015.withColumn('hour_dropoff',hour(df2015['tpep_dropoff_datetime']))
df2015 = df2015.withColumn('date_pickup',date_format(df2015['tpep_pickup_datetime'],'yyyy-MM-dd'))
df2015 = df2015.withColumn('date_dropoff',date_format(df2015['tpep_dropoff_datetime'],'yyyy-MM-dd'))
df2015.show()

In [ ]:
#average number of dropoffs in the area during that hour the day
avg_dropoffs = df2015.groupby('dropoff_zone','hour_dropoff','date_dropoff').count()
avg_dropoffs.show()

In [ ]:
avg_dropoffs = avg_dropoffs.groupby('dropoff_zone','hour_dropoff').agg({'count':'mean'}).toPandas()
avg_dropoffs.head()

In [ ]:
steps = []
datas = []



for hour in range(24):
    t = avg_dropoffs[avg_dropoffs['hour_dropoff'] == hour]
    data = go.Choroplethmapbox(
        geojson = nyc_geo,
        featureidkey="properties.zone",
        locations = t['dropoff_zone'],
        z = t['avg(count)'],
        zauto=False,
        colorscale='ylorrd',
        marker={'opacity':0.8,'line_width':0.5},
        hovertext = t['dropoff_zone'],
        hoverinfo='text + z',
        showscale=True
    )
    
    datas.append(data)
    
    visibles = [False]*24
    visibles[hour] = True
    
    step = {
        "args": [{"visible":visibles}],
        "label": hour+1,
        "method": "restyle"
    }
    
    steps.append(step)
    

sliders = [{"x":0,"y":0.15,"pad":{"t":24},"steps":steps,"active":1,"currentvalue":{"prefix":"hour_dropoff"}}]
    


avg_dropoffs_fig = go.Figure(
    data = datas
)

avg_dropoffs_fig.update_layout(
    sliders = sliders,
    mapbox = {'accesstoken':token,'center':{'lon':-73.965691,'lat':40.97},'zoom':5},margin = {'l':1,'r':1,'t':1,'b':1})
avg_dropoffs_fig.show()

In [ ]:
#average ratio of tip over total fare amount for pickups in the area at given hour of the day

ratio_pickup_fare = df2015.groupby('pickup_zone','hour_pickup').agg(sum(df2015['tip_amount']),sum(df2015['total_amount']))
ratio_pickup_fare.show()

In [ ]:
avg_ratio_pickup_fare = ratio_pickup_fare.withColumn('ratio',ratio_pickup_fare['sum(tip_amount)']/ratio_pickup_fare['sum(total_amount)']).select('pickup_zone','hour_pickup','ratio')
avg_ratio_pickup_fare.show()

In [ ]:
avg_ratio_pickup_fare = avg_ratio_pickup_fare.toPandas()

steps = []
datas = []



for hour in range(24):
    t = avg_ratio_pickup_fare[avg_ratio_pickup_fare['hour_pickup'] == hour]
    data = go.Choroplethmapbox(
        geojson = nyc_geo,
        featureidkey="properties.zone",
        locations = t['pickup_zone'],
        z = t['ratio'],
        zauto=False,
        colorscale='ylorrd',
        marker={'opacity':0.8,'line_width':0.5},
        hovertext = t['pickup_zone'],
        hoverinfo='text + z',
        showscale=True
    )
    
    datas.append(data)
    
    visibles = [False]*24
    visibles[hour] = True
    
    step = {
        "args": [{"visible":visibles}],
        "label": hour+1,
        "method": "restyle"
    }
    
    steps.append(step)
    

sliders = [{"x":0,"y":0.15,"pad":{"t":24},"steps":steps,"active":1,"currentvalue":{"prefix":"hour_pickup"}}]
    


avg_pickup_fig = go.Figure(
    data = datas
)

avg_pickup_fig.update_layout(
    sliders = sliders,
    mapbox = {'accesstoken':token,'center':{'lon':-73.965691,'lat':40.97},'zoom':5},margin = {'l':1,'r':1,'t':1,'b':1})
avg_pickup_fig.show()